In [5]:
import subprocess
from pyspark.sql.types import IntegerType,FloatType
from functools import reduce
from pyspark.sql import DataFrame,SparkSession
from pyspark.sql.functions import sum
spark=SparkSession.builder.appName('check').config("spark.some.config.option","some-value").master('hdfs').getOrCreate()

In [6]:
def hdfs_file_read(file):
    rdd = spark.read.csv(file,header=True,inferSchema = True)
    # rdd=rdd.drop("axlespacings", "axleweights", "numberofaxles","validitycode","duration","temperature","weight","hour","minute","second","millisecond")
    return rdd

In [7]:
def get_Dataframe(x):
    p=subprocess.Popen(x,
    shell=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT)
    result=list()
    for line in p.stdout.readlines():
        file = line.decode().strip()

        df= hdfs_file_read(file)
        df= df.withColumn("speed", df["speed"].cast(FloatType()))

        check_columns=df.select('class','classname').distinct().sort('class',ascending=True)
    #     check_columns.show()
        #remove the null rows
        speed_df=df.na.drop(subset=['classname'])
    #     speed_df.select('class','classname').distinct().sort('class',ascending=True).show()

        speed_df=speed_df.groupBy('class','classname').avg('speed').sort('class',ascending=True)
        speed_df=speed_df.withColumnRenamed("avg(speed)", "speed")
    #     speed_df.show()
        result.append(speed_df)

    union_df = reduce(DataFrame.unionAll, result)
    union_df=union_df.groupBy('class','classname').avg('speed').sort('class',ascending=True)
    union_df.show()
    return union_df


In [8]:
w1="hdfs dfs -ls hdfs://localhost:9000/user/akash/trafficCount/processedVehicleData| awk '{print $8}' | awk 'NR!=1' "
w2="hdfs dfs -ls hdfs://localhost:9000/user/akash/trafficCount/lockdownProcessedVehicleData| awk '{print $8}' | awk 'NR!=1' "
L=[w1,w2]
for i in range(0,2):
        j=str(i)
        globals()[f"week{j}"] =  get_Dataframe(L[i])
        
week0.show()
week1.show()
# week0.createOrReplaceTempView("normalWeek")

# week1.createOrReplaceTempView("lockdownWeek")

+-----+---------+-----------------+
|class|classname|       avg(speed)|
+-----+---------+-----------------+
|    1|    MBIKE|85.87613161095074|
|    2|      CAR|92.29858413944116|
|    3|      LGV|94.26504522661584|
|    4|      BUS|82.54091919314335|
|    5|  HGV_RIG|83.52102209610372|
|    6|  HGV_ART|83.38206595764122|
|    7|  CARAVAN|83.13369684835732|
+-----+---------+-----------------+

+-----+---------+-----------------+
|class|classname|       avg(speed)|
+-----+---------+-----------------+
|    1|    MBIKE|90.48422177807804|
|    2|      CAR|96.77241797549061|
|    3|      LGV|95.94105077135649|
|    4|      BUS|77.88388952490064|
|    5|  HGV_RIG|82.46460160583159|
|    6|  HGV_ART|85.23505061315497|
|    7|  CARAVAN|84.21743287793122|
+-----+---------+-----------------+

+-----+---------+-----------------+
|class|classname|       avg(speed)|
+-----+---------+-----------------+
|    1|    MBIKE|85.87613161095074|
|    2|      CAR|92.29858413944116|
|    3|      LGV|94.265045